In [ ]:
# import tensorflow 
# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import to_categorical

# # Load the CIFAR-10 dataset
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()

# # Normalize the pixel values to be between 0 and 1
# x_train = x_train / 255.0
# x_test = x_test / 255.0

# # Convert the labels to one-hot encoded vectors
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# # Create a convolutional neural network
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(10, activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model on the training data
# model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# # Evaluate the model on the testing data
# loss, accuracy = model.evaluate(x_test, y_test)

# print(f"Test accuracy: {accuracy}")

In [ ]:
# def func(a):								
#     for i in range(n):
#         a=np.random.randn(n)
#         b=np.random.randn(n)
#         c=np.dot(a,b)

# @jit(target_backend='cuda')
# def func2(a):
#     for i in range(n):
#         A=np.random.randn(n)
#         B=np.random.randn(n)
#         C=np.dot(A,B)

# if __name__=="__main__":
#     n=100000
#     a=np.random.randn(n)
#     # print(a)
#     start = timer()
#     func(a)
#     print("without GPU:", timer()-start)	
	
#     start = timer()
#     func2(a)
#     print("with GPU:", timer()-start)

In [2]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer

# normal function to run on cpu
def func(a):								
	for i in range(1000000):
		a[i]+= 1	

# function optimized to run on gpu
@jit(target_backend='cuda')						
def func2(a):
	for i in range(1000000):
		a[i]+= 1
if __name__=="__main__":
	n = 1000000		
	a = np.ones(n, dtype = np.float64)
	
	start = timer()
	func(a)
	print("without GPU:", timer()-start)	
	
	start = timer()
	func2(a)
	print("with GPU:", timer()-start)


C:\Users\matan.s\AppData\Local\Temp\ipykernel_8472\4086339893.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


without GPU: 0.09785790000023553
with GPU: 2.7425382999999783


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun 15 12:04:29 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA T1000 8GB             WDDM  | 00000000:01:00.0  On |                  N/A |
| 40%   48C    P8              N/A /  50W |   7858MiB /  8192MiB |     40%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
name = "bigscience/bloom-3b"
text = "Hello my name is"
max_new_tokens = 20

def generate_from_model(model, tokenizer):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_8bit = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(name)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: Loading binary c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable
CUDA SETUP: Problem: The main issue seems to be that the main CUDA library was not detected.
CUDA SETUP: Solution 1): Your paths are probably not up-to-date. You can update them via: sudo ldconfig.
CUDA SETUP: Solution 2): If you do not have sudo rights, you can do the following:
CUDA SETUP: Solution 2a): Find the cuda library via: find / -name libcuda.so 2>/dev/null
CUDA SETUP: Solution 2b): Once the library is found add it to the LD_LIBRARY_PATH: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:FOUND_PATH_FROM_2a
CUDA SETUP: Solution 2c): For a permanent solution add the export from 2b in

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 from transformers import AutoModelForCausalLM, AutoTokenizer                                 │
│   2 import torch                                                                                 │
│   3                                                                                              │
│ ❱ 4 model_8bit = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=     │
│   5 tokenizer = AutoTokenizer.from_pretrained(name)                                              │
│   6                                                                                              │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\models\auto\auto_factory.p │
│ y:467 in from_pretrained                                                                         │
│                                                                                                  │
│   464 │   │   │   )                                                                              │
│   465 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   466 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 467 │   │   │   return model_class.from_pretrained(                                            │
│   468 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   469 │   │   │   )                                                                              │
│   470 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\modeling_utils.py:2629 in  │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   2626 │   │   │   keep_in_fp32_modules = []                                                     │
│   2627 │   │                                                                                     │
│   2628 │   │   if load_in_8bit:                                                                  │
│ ❱ 2629 │   │   │   from .utils.bitsandbytes import get_keys_to_not_convert, replace_8bit_linear  │
│   2630 │   │   │                                                                                 │
│   2631 │   │   │   load_in_8bit_skip_modules = quantization_config.llm_int8_skip_modules         │
│   2632 │   │   │   load_in_8bit_threshold = quantization_config.llm_int8_threshold               │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\utils\bitsandbytes.py:9 in │
│ <module>                                                                                         │
│                                                                                                  │
│     6                                                                                            │
│     7                                                                                            │
│     8 if is_bitsandbytes_available():                                                            │
│ ❱   9 │   import bitsandbytes as bnb                                                             │
│    10 │   import torch                                     

In [7]:
generate_from_model(model_8bit, tokenizer)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 generate_from_model(model_8bit, tokenizer)                                                   │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model_8bit' is not defined

In [2]:


from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

name = "bigscience/bloom-3b"
text = "Hello my name is"
max_new_tokens = 20


tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForCausalLM.from_pretrained(name)
pipe = pipeline(model=model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)
output = pipe(text)
print(output)



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7                                                                                             │
│    8 tokenizer = AutoTokenizer.from_pretrained(name)                                             │
│    9 model = AutoModelForCausalLM.from_pretrained(name)                                          │
│ ❱ 10 pipe = pipeline(model=model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)            │
│   11 output = pipe(text)                                                                         │
│   12 print(output)                                                                               │
│   13                                                                                             │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\pipelines\__init__.py:722  │
│ in pipeline                                                                                      │
│                                                                                                  │
│   719 │                                                                                          │
│   720 │   if task is None and model is not None:                                                 │
│   721 │   │   if not isinstance(model, str):                                                     │
│ ❱ 722 │   │   │   raise RuntimeError(                                                            │
│   723 │   │   │   │   "Inferring the task automatically requires to check the hub with a model   │
│   724 │   │   │   │   f"{model} is not a valid model_id."                                        │
│   725 │   │   │   )                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Inferring the task automatically requires to check the hub with a model_id defined as a 
`str`.BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=2560, out_features=250880, bias=False)
) is not a valid model_id.

In [5]:
import numpy as np
from datasets import Dataset


seq_len, dataset_size = 512, 512
dummy_data = {
    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
    "labels": np.random.randint(0, 1, (dataset_size)),
}
ds = Dataset.from_dict(dummy_data)
ds.set_format("pt")

In [6]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [7]:
print_gpu_utilization()

GPU memory occupied: 916 MB.


In [8]:
import torch


torch.ones((1, 1)).to("cuda")
print_gpu_utilization()

GPU memory occupied: 975 MB.


In [9]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased").to("cuda")
print_gpu_utilization()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly

GPU memory occupied: 2265 MB.


In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA T1000 8GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [26]:
# benchmark_pytorch.py
from timeit import default_timer as timer
import torch
import torch.nn as nn
import torchvision
import torch.utils.benchmark as benchmark


@torch.no_grad()
def measure_time_host(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start = timer()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        if synchronize:
            torch.cuda.synchronize()
        end = timer()
        elapsed_time_ms = (end - start) * 1000

    else:
        for _ in range(num_repeats):
            start = timer()
            _ = model.forward(input_tensor)
            if synchronize:
                torch.cuda.synchronize()
            end = timer()
            elapsed_time_ms += (end - start) * 1000

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def measure_time_device(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        end_event.record()
        if synchronize:
            # This has to be synchronized to compute the elapsed time.
            # Otherwise, there will be runtime error.
            torch.cuda.synchronize()
        elapsed_time_ms = start_event.elapsed_time(end_event)

    else:
        for _ in range(num_repeats):
            start_event = torch.cuda.Event(enable_timing=True)
            end_event = torch.cuda.Event(enable_timing=True)
            start_event.record()
            _ = model.forward(input_tensor)
            end_event.record()
            if synchronize:
                # This has to be synchronized to compute the elapsed time.
                # Otherwise, there will be runtime error.
                torch.cuda.synchronize()
            elapsed_time_ms += start_event.elapsed_time(end_event)

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def run_inference(model: nn.Module,
                  input_tensor: torch.Tensor) -> torch.Tensor:

    return model.forward(input_tensor)


def main() -> None:

    num_warmups = 100
    num_repeats = 1000
    input_shape = (1, 3, 224, 224)

    device = torch.device("cuda:0")

    # model = torchvision.models.resnet18(pretrained=False)
    model = nn.Conv2d(in_channels=input_shape[1],
                      out_channels=256,
                      kernel_size=(5, 5))

    model.to(device)
    model.eval()

    # Input tensor
    input_tensor = torch.rand(input_shape, device=device)

    torch.cuda.synchronize()

    print("Latency Measurement Using CPU Timer...")
    for continuous_measure in [True, False]:
        for synchronize in [True, False]:
            try:
                latency_ms = measure_time_host(
                    model=model,
                    input_tensor=input_tensor,
                    num_repeats=num_repeats,
                    num_warmups=num_warmups,
                    synchronize=synchronize,
                    continuous_measure=continuous_measure,
                )
                print(f"|"
                      f"Synchronization: {synchronize!s:5}| "
                      f"Continuous Measurement: {continuous_measure!s:5}| "
                      f"Latency: {latency_ms:.5f} ms| ")
            except Exception as e:
                print(f"|"
                      f"Synchronization: {synchronize!s:5}| "
                      f"Continuous Measurement: {continuous_measure!s:5}| "
                      f"Latency: N/A     ms| ")
            torch.cuda.synchronize()

    print("Latency Measurement Using CUDA Timer...")
    for continuous_measure in [True, False]:
        for synchronize in [True, False]:
            try:
                latency_ms = measure_time_device(
                    model=model,
                    input_tensor=input_tensor,
                    num_repeats=num_repeats,
                    num_warmups=num_warmups,
                    synchronize=synchronize,
                    continuous_measure=continuous_measure,
                )
                print(f"|"
                      f"Synchronization: {synchronize!s:5}| "
                      f"Continuous Measurement: {continuous_measure!s:5}| "
                      f"Latency: {latency_ms:.5f} ms| ")
            except Exception as e:
                print(f"|"
                      f"Synchronization: {synchronize!s:5}| "
                      f"Continuous Measurement: {continuous_measure!s:5}| "
                      f"Latency: N/A     ms| ")
            torch.cuda.synchronize()

    print("Latency Measurement Using PyTorch Benchmark...")
    num_threads = 1
    timer = benchmark.Timer(stmt="run_inference(model, input_tensor)",
                            setup="from __main__ import run_inference",
                            globals={
                                "model": model,
                                "input_tensor": input_tensor
                            },
                            num_threads=num_threads,
                            label="Latency Measurement",
                            sub_label="torch.utils.benchmark.")

    profile_result = timer.timeit(num_repeats)
    # https://pytorch.org/docs/stable/_modules/torch/utils/benchmark/utils/common.html#Measurement
    print(f"Latency: {profile_result.mean * 1000:.5f} ms")


if __name__ == "__main__":

    main()

Latency Measurement Using CPU Timer...
|Synchronization: True | Continuous Measurement: True | Latency: 2.72778 ms| 
|Synchronization: False| Continuous Measurement: True | Latency: 0.01405 ms| 
|Synchronization: True | Continuous Measurement: False| Latency: 2.79816 ms| 
|Synchronization: False| Continuous Measurement: False| Latency: 0.01215 ms| 
Latency Measurement Using CUDA Timer...
|Synchronization: True | Continuous Measurement: True | Latency: 2.77255 ms| 
|Synchronization: False| Continuous Measurement: True | Latency: N/A     ms| 
|Synchronization: True | Continuous Measurement: False| Latency: 2.70020 ms| 
|Synchronization: False| Continuous Measurement: False| Latency: N/A     ms| 
Latency Measurement Using PyTorch Benchmark...
Latency: 2.76667 ms


In [1]:
import torch
import time

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Create a large tensor
n=20000
x = torch.randn(n, n).to(device)

# Measure the time to perform a matrix multiplication on the CPU
start_time = time.time()
y = torch.matmul(x, x)
end_time = time.time()
cpu_time = end_time - start_time
print(f"CPU time: {cpu_time:.2f} seconds")

# Measure the time to perform a matrix multiplication on the GPU
start_time = time.time()
y = torch.matmul(x, x).to(device)
torch.cuda.synchronize()
end_time = time.time()
gpu_time = end_time - start_time
print(f"GPU time: {gpu_time:.2f} seconds")

# Calculate the speedup
speedup = cpu_time / gpu_time
print(f"Speedup: {speedup:.2f}x")

Device: cuda
CPU time: 0.50 seconds
GPU time: 20.63 seconds
Speedup: 0.02x
